In [ ]:
# pip install openpyxl


In [3]:
import pandas as pd
import numpy as np
import os


In [ ]:
# read dataset
# df = pd.read_excel('your_file.xlsx')
# Orderline dataset
df1 = pd.read_csv('Orderlijnen 01112022-30112022.csv', low_memory=False)

# Bundle Item data
df2 = pd.read_excel('Bundle_item_TE.xlsx')

# Item Overview data
df3 = pd.read_excel('Item_overview_TE.xlsx')




In [ ]:
df1.head()

In [ ]:
df3.head()

In [ ]:
columns_to_copy = ["Order_date", "Nb_of_colli", "Nb_of_pallets",	"Article_number"	,"Product_number"	,"Qty_actual"]
df1_new = df1[columns_to_copy]
df1_new.head()

In [ ]:
# Value to search for values
# Create a new column in df1 to store the extracted values
# df1_new['ChildQty'] = df1['Product_number'].map(df2.set_index('ProductCode')['ChildQty'])

# Use merge to match and append values from df2 to df1
merged_df = df1_new.merge(df2, left_on='Product_number', right_on='ProductCode', how='left')

merged_df.head()

In [ ]:
drop_col = [6,7,8,9,10,11,12,13,15,16,17,18,19]

merged_df = merged_df.drop(merged_df.columns[drop_col], axis=1)
merged_df.head()

In [ ]:
# Use merge to match and append values from df2 to df1
merged_df_new = merged_df.merge(df3, left_on='Product_number', right_on='Item No.', how='left')

merged_df_new.head()

In [ ]:
drop_col = [7,8,9,10,11,13]

merged_df_new = merged_df_new.drop(merged_df_new.columns[drop_col], axis=1)
merged_df_new.head()

In [ ]:
# Full case, Single unit calculation and separation of automatic from manual

def full_case_calculation(df):
    # Calculation for the full case
    df['Full_case'] = np.where(df['ChildQty'].notna() & df['Qty_actual'].notna(),
                              df['Qty_actual'] // df['ChildQty'], 0)

    # Calculation for the single unit
    df['Single_unit'] = np.where(df['ChildQty'].notna() & df['Qty_actual'].notna(),
                                df['Qty_actual'] % df['ChildQty'], 0)

    # Set 'Single_unit' to 'Qty_actual' where 'Strategy Code' is NaN or not 'HB_OSR' or 'HB_OSR_TMB'
    df.loc[df['Strategy Code'].isna() | ~df['Strategy Code'].isin(["HB_OSR", "HB_OSR_TMB"]), 'Single_unit'] = df['Qty_actual']

    # Create the 'Automated' column based on 'Strategy Code'
    # 1 if Strategy Code is "HB_OSR", "HB_OSR_TMB"
    # 0 otherwise
    df['Automated'] = df['Strategy Code'].isin(["HB_OSR", "HB_OSR_TMB"]).astype(int)

    return df




In [ ]:
first_result = full_case_calculation(merged_df_new)

# def update_single_unit(row):
#     if pd.isna(row['Strategy Code']) and row['Strategy Code'] not in ["HB_OSR", "HB_OSR_TMB"]:
#         return row['Qty_actual']
#     return row['Single_unit']

# first_result['Single_unit'] = first_result.apply(update_single_unit, axis=1)
first_result.tail(100)

In [ ]:
# Some values in Item overview not in Bundle has NAN
# That is they dont have a full case therefore replace Single_unit with the values of Qty_actual

first_result.loc[first_result['ChildQty'].isna(), 'Single_unit'] = first_result['Qty_actual']

first_result.tail(20)

In [ ]:
# Calculate the workload for each date

def calculate_time(row):
    if row['Automated'] == 1:
        single_unit_time = (row['Single_unit'] * 60) / 200 / 60
        return single_unit_time
    else:
        x_i_time = (row['Full_case'] * 60) / 80 / 60
        y_i_time = (row['Single_unit'] * 60) / 40 / 60
        return x_i_time + y_i_time

# Apply the function to create a new 'Time' column
first_result['Time1(hours)'] = first_result.apply(calculate_time, axis=1)

first_result.tail()

In [ ]:
# Check specific date

# first_result[first_result['Order_date'] == "10/13/2022"]

In [ ]:
# Aggregate the total number of hours by grouping the Order_date

# Convert the 'Date' column to a datetime object
# first_result['Order_date'] = pd.to_datetime(first_result['Order_date'], format='%d/%m/%Y')

# Group the data by date and calculate the sum of the 'Time' column
second_result = first_result.groupby('Order_date')['Time1(hours)'].sum().reset_index()
second_result

In [ ]:
sorted_result = second_result.sort_values(by='Order_date', ascending=False)
sorted_result

In [4]:
# Merge all the excel files

dataframes = []

directory = 'C:\\Users\\harve\\Downloads\\Orderline_dataset\\'

for filename in os.listdir(directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"): 
        filepath = os.path.join(directory, filename)
        df = pd.read_excel(filepath)
        dataframes.append(df)


# Merge the file
merged_df = pd.concat(dataframes, ignore_index=True)


# export the dataframe to excel file

# merged_df.to_excel('merged_file.xlsx', index=False)


In [9]:
# Bundle Item data
df2 = pd.read_excel('Bundle_item_TE.xlsx')

# Item Overview data
df3 = pd.read_excel('Item_overview_TE.xlsx')

In [5]:
merged_df.head()

,Order_number,Initiator_code,Order_date,Order_date_time,Order_wms_entry_date,Processed_date,Erp_error,Ship_date,Preferred_ship_date,Delay,...,Pharmacie des Plantes sa,13:00:00,B. One hour before cut-off time,BPSRO,2192938,1142264,Steovit Forte Sinaas1000mg/880IE - 30 Bruistabl,6,6.1,1173873
0,01000100,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01000101,KP,2022-01-04,11:49:04,2022-01-04,2022-01-04,1.0,2022-01-04 00:00:00,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Dimension of the dataset

merged_df.shape

(1307820, 75)

In [67]:
columns_to_copy = ["Order_number","Order_date", "Nb_of_colli", "Nb_of_pallets",	"Article_number"	,"Product_number"	,"Qty_actual"]
real_df1 = merged_df[columns_to_copy]
real_df1.head()


,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0


In [68]:
# Use merge to match and append values from df2 to df1
real_df2 = real_df1.merge(df2, left_on='Product_number', right_on='ProductCode', how='left')

real_df2.head()

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,Host,HostArtikelCode,ProductCode,...,ProductGroep,BundelType,ChildBundelType,Bundelbeschrijving,ChildQty,heigth,width,length,weight,Omgeving
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
drop_col = [7,8,9,10,11,12,13,14,16,17,18,19,20]

real_df2 = real_df2.drop(real_df2.columns[drop_col], axis=1)


In [70]:
real_df2.head()

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN


In [71]:
# Use merge to match and append values from df2 to df1
real_df3 = real_df2.merge(df3, left_on='Product_number', right_on='Item No.', how='left')

real_df3.head()

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Host ID,Item No.,Catalogue Code,Description,Item Grp.,Strategy Code,Pickmethod,Environ. Code
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
drop_col = [8,9,10,11,12,14]

real_df3 = real_df3.drop(real_df3.columns[drop_col], axis=1)
real_df3.head()

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN


In [73]:
unique_child_qty_values = real_df3["Strategy Code"].unique()
print(unique_child_qty_values)


[nan 'HB_OSR' 'HB_MTE' 'OF_MW' 'OF_ADR' 'HB_OSR_TMB' 'OF_FRIGO' 'AS_FRIGO'
 'DAM_TE' '2K_FRB_M' '2K_FRB_F' '2K_FR_CYTO' '2K_FRIGO_S' '2K_FRIGO_F'
 '2K_FRB_S' '2K_FRIGO_M' '2L_FRIGO_M' '2L_FRIGO_F' '2L_FRB_M' '2L_FRB_F'
 '2L_FR_CYTO' '2L_FRIGO_S' '2L_FRB_S' 'LB_SSS' 'LN_FRIGO_S' 'LB_FR_SSS'
 'LN_SSS' 'LB_FRIGO_S' 'HB_MTE_CYT' '02_RET' 'LN_FR_SSS' '02_FRIGO'
 'RET_TE' 'HB_TMB' '02_MW' '98_DES' 'EG_CYT_ADR' 'EG_MW' 'DEFAULT'
 'EG_FRIGO_S' 'EG_TE_ADR' 'EG_FR_CYT' 'EG_TE_NAR' 'AS_MW' 'BX_MW' 'HERCON'
 'HB_MTE_GVP' 'TEMPORARY' 'BX_Z_ALG' 'HB_BP' 'HB' 'VA_FRB_M']


In [78]:
# Full case, Single unit calculation and separation of automatic from manual

import numpy as np

def full_case_calculation(df):
    # Input parameter validation
    required_columns = ['ChildQty', 'Qty_actual', 'Strategy Code']
    missing_columns = set(required_columns) - set(df.columns)
    
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

    # Calculation for the full case
    df['Full_case'] = np.where(df['ChildQty'].notna() & df['Qty_actual'].notna(),
                               df['Qty_actual'] // df['ChildQty'], 0)

    # Calculation for the single unit
    df['Single_unit'] = np.where(df['ChildQty'].notna() & df['Qty_actual'].notna(),
                                 df['Qty_actual'] % df['ChildQty'], 0)

    # Set 'Single_unit' to 'Qty_actual' where 'Strategy Code' is NaN
    df['Single_unit'] = np.where(df['ChildQty'].isna(),
                                 df['Qty_actual'], df['Single_unit'])

    # Create the 'Automated' column based on 'Strategy Code'
    # 1 if Strategy Code is "HB_OSR", "HB_OSR_TMB"
    # 0 otherwise
    df['Automated'] = df['Strategy Code'].isin(["HB_OSR", "HB_OSR_TMB"]).astype(int)

    return df


In [ ]:
# Calculate the workload for each date

def calculate_time(row):
    if row['Automated'] == 1:
        single_unit_time = (row['Single_unit'] * 60) / 200 / 60
        return single_unit_time
    else:
        x_i_time = (row['Full_case'] * 60) / 80 / 60
        y_i_time = (row['Single_unit'] * 60) / 40 / 60
        return x_i_time + y_i_time

# Apply the function to create a new 'Time' column
# first_result['Time1(hours)'] = first_result.apply(calculate_time, axis=1)


In [79]:
first_result1 = full_case_calculation(real_df3)

first_result1

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN,0.0,800.0,0
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN,0.0,6.0,0
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN,0.0,6.0,0
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN,0.0,6.0,0
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN,0.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344089,99000484,2022-12-16,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344090,99000485,2022-12-20,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344091,99000488,2022-12-27,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344092,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0


In [125]:
Not_OSR = first_result1[first_result1["Automated"]==0]

OSR = first_result1[first_result1["Automated"]==1]

In [128]:
Not_OSR

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN,0.0,800.0,0
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN,0.0,6.0,0
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN,0.0,6.0,0
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN,0.0,6.0,0
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN,0.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344089,99000484,2022-12-16,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344090,99000485,2022-12-20,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344091,99000488,2022-12-27,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344092,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0


In [112]:
# OSR Manual picking PIE
new_OSR = OSR[OSR["Full_case"]!=1]

In [95]:
new_OSR

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated
125,03012201,2022-01-03,1.0,0.0,3181781,6105054,3.0,NaN,HB_OSR,B,0.0,3.0,1
126,03012201,2022-01-03,1.0,0.0,0052589,1138529,5.0,NaN,HB_OSR,B,0.0,5.0,1
127,03012202,2022-01-03,4.0,0.0,3181807,6105057,1.0,NaN,HB_OSR,B,0.0,1.0,1
128,03012202,2022-01-03,4.0,0.0,3181849,6105059,1.0,NaN,HB_OSR,B,0.0,1.0,1
131,03012202,2022-01-03,4.0,0.0,3181781,6105054,3.0,NaN,HB_OSR,B,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344061,98015198,2022-12-29,5.0,0.0,2983799,1142250,240.0,NaN,HB_OSR,B,0.0,240.0,1
1344062,98015199,2022-12-29,5.0,0.0,2983799,1142250,240.0,NaN,HB_OSR,B,0.0,240.0,1
1344063,98015200,2022-12-29,0.0,0.0,2983799,1142250,0.0,NaN,HB_OSR,B,0.0,0.0,1
1344064,98015201,2022-12-29,0.0,0.0,2983799,1142250,0.0,NaN,HB_OSR,B,0.0,0.0,1


In [113]:
# Drop column 4,5,6,7,8,9,10,12
drop_cols = [4,5,6,7,8,9,10,12]
new_OSR = new_OSR.drop(new_OSR.columns[drop_cols], axis=1)

# Remove rows where both 'Nb_of_colli' and 'B' are 0
new_OSR = new_OSR[(new_OSR['Nb_of_colli'] != 0) | (new_OSR['Nb_of_pallets'] != 0)]

In [99]:
new_OSR.head()

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Single_unit
125,03012201,2022-01-03,1.0,0.0,3.0
126,03012201,2022-01-03,1.0,0.0,5.0
127,03012202,2022-01-03,4.0,0.0,1.0
128,03012202,2022-01-03,4.0,0.0,1.0
131,03012202,2022-01-03,4.0,0.0,3.0


In [114]:
new_OSR_agg = new_OSR.groupby(['Order_number', 'Order_date']).agg(Count=('Single_unit', 'count')).reset_index()

# drop the Order number
new_OSR_agg = new_OSR_agg.drop(new_OSR_agg.columns[[0]],axis=1)
new_OSR_agg

,Order_date,Count
0,2023-02-01,2
1,2023-02-01,1
2,2023-02-01,1
3,2023-02-01,3
4,2023-02-01,2
...,...,...
139591,2023-07-31,19
139592,2023-07-31,12
139593,2023-07-31,11
139594,2023-07-31,26


In [107]:
# Keep track
# new_OSR_agg[new_OSR_agg["Order_date"]=="2022-01-03"]

,Order_date,Count
106,2022-01-03,2
107,2022-01-03,8
108,2022-01-03,26
109,2022-01-03,1
110,2022-01-03,3
...,...,...
5093,2022-01-03,12
5094,2022-01-03,21
5095,2022-01-03,13
5096,2022-01-03,20


In [116]:
# Sum the counts by unique dates
new_OSR_time = new_OSR_agg.groupby('Order_date')['Count'].sum().reset_index(name='sum_count')
new_OSR_time['time']  = (new_OSR_time['sum_count'] * 60) / 200 / 60
new_OSR_time.head()

,Order_date,sum_count,time
0,2022-01-01,4,0.020
1,2022-01-03,712,3.560
2,2022-01-04,497,2.485
3,2022-01-05,669,3.345
4,2022-01-06,527,2.635
...,...,...,...
454,2023-07-26,1601,8.005
455,2023-07-27,2650,13.250
456,2023-07-28,2462,12.310
457,2023-07-29,71,0.355


In [ ]:
# Drop column 4,5,6,7,8,9,10,12
# Drop columns with Nb_of_colli & Nb_of_pallets == 0
# Count single unit according to their lines
# Sum the counts by the unique date
# calculate time

In [129]:
Not_OSR

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Article_number,Product_number,Qty_actual,ChildQty,Strategy Code,Environ. Code,Full_case,Single_unit,Automated
0,01000100,2022-01-04,20.0,0.0,4380564,5407009330063,800.0,NaN,NaN,NaN,0.0,800.0,0
1,01000101,2022-01-04,1.0,0.0,4393997,5407009330391,6.0,NaN,NaN,NaN,0.0,6.0,0
2,01000101,2022-01-04,1.0,0.0,4394060,5407009330469,6.0,NaN,NaN,NaN,0.0,6.0,0
3,01000101,2022-01-04,1.0,0.0,4394094,5407009330490,6.0,NaN,NaN,NaN,0.0,6.0,0
4,01000101,2022-01-04,1.0,0.0,4380564,5407009330063,12.0,NaN,NaN,NaN,0.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344089,99000484,2022-12-16,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344090,99000485,2022-12-20,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344091,99000488,2022-12-27,0.0,0.0,3021508,6083748,0.0,NaN,OF_FRIGO,F,0.0,0.0,0
1344092,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0


In [142]:
Not_OSR_full = Not_OSR[Not_OSR["Full_case"]==1] #10

Not_OSR_single = Not_OSR[Not_OSR["Full_case"]==0] #11

In [144]:
drop_cols = [4,5,6,7,8,9,11,12]
Not_OSR_full = Not_OSR_full.drop(Not_OSR_full.columns[drop_cols], axis=1)
new_Not_OSR_full = Not_OSR_full[(Not_OSR_full['Nb_of_colli'] != 0) | (Not_OSR_full['Nb_of_pallets'] != 0)]
new_Not_OSR_full



,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Full_case
1725,20220103003,2022-01-03,2.0,0.0,1.0
1726,20220103004,2022-01-03,2.0,0.0,1.0
1784,20220105004,2022-01-05,1.0,0.0,1.0
1842,20220106018,2022-01-06,7.0,0.0,1.0
1907,20220106027,2022-01-06,2.0,0.0,1.0
...,...,...,...,...,...
1342029,98001488,2022-12-19,2.0,0.0,1.0
1342048,98001489,2022-12-19,28.0,0.0,1.0
1342062,98001489,2022-12-19,28.0,0.0,1.0
1342064,98001489,2022-12-19,28.0,0.0,1.0


In [147]:
drop_cols = [4,5,6,7,8,9,10,12]
# Not_OSR_single = Not_OSR_single.drop(Not_OSR_single.columns[drop_cols], axis=1)

# Remove rows where both 'Nb_of_colli' and 'B' are 0
new_Not_OSR_single = Not_OSR_single[(Not_OSR_single['Nb_of_colli'] != 0) | (Not_OSR_single['Nb_of_pallets'] != 0)]
new_Not_OSR_single

,Order_number,Order_date,Nb_of_colli,Nb_of_pallets,Single_unit
0,01000100,2022-01-04,20.0,0.0,800.0
1,01000101,2022-01-04,1.0,0.0,6.0
2,01000101,2022-01-04,1.0,0.0,6.0
3,01000101,2022-01-04,1.0,0.0,6.0
4,01000101,2022-01-04,1.0,0.0,12.0
...,...,...,...,...,...
1344070,99000458,2022-12-02,1.0,0.0,12.0
1344071,99000459,2022-12-05,1.0,0.0,12.0
1344072,99000461,2022-12-08,1.0,0.0,6.0
1344092,NaN,NaT,NaN,NaN,NaN


In [149]:
new_Not_OSR_agg_full = new_Not_OSR_full.groupby(['Order_number', 'Order_date']).agg(Count=('Full_case', 'count')).reset_index()

new_Not_OSR_agg_single = new_Not_OSR_single.groupby(['Order_number', 'Order_date']).agg(Count=('Single_unit', 'count')).reset_index()


In [152]:
# new_Not_OSR_agg_full

In [153]:
# drop the Order number
new_Not_OSR_agg1 = new_Not_OSR_agg_full.drop(new_Not_OSR_agg_full.columns[[0]],axis=1)

new_Not_OSR_agg2 = new_Not_OSR_agg_single.drop(new_Not_OSR_agg_single.columns[[0]],axis=1)

In [155]:
# Sum the counts by unique dates
new_Not_OSR_time_full = new_Not_OSR_agg1.groupby('Order_date')['Count'].sum().reset_index(name='sum_count')

new_Not_OSR_time_single = new_Not_OSR_agg2.groupby('Order_date')['Count'].sum().reset_index(name='sum_count')

new_Not_OSR_time_single['time']  = (new_Not_OSR_time_single['sum_count'] * 60) / 40 / 60
new_Not_OSR_time_full['time']  = (new_Not_OSR_time_full['sum_count'] * 60) / 80 / 60

In [168]:
new_Not_OSR_time_single

,Order_date,sum_count,time
0,2021-01-27,14,0.350
1,2021-03-11,1,0.025
2,2022-01-01,1,0.025
3,2022-01-03,834,20.850
4,2022-01-04,541,13.525
...,...,...,...
420,2023-07-26,707,17.675
421,2023-07-27,765,19.125
422,2023-07-28,803,20.075
423,2023-07-29,7,0.175


In [178]:
# Merge the dataframes based on the 'date' column
combined_time = pd.merge(new_OSR_time, new_Not_OSR_time_full, on='Order_date')

# combined_time = pd.merge(combined_time, new_Not_OSR_time_single, on='Order_date')

In [179]:
combined_time['sum_time'] = combined_time['time_x'] + combined_time['time_y']

combined_time

,Order_date,sum_count_x,time_x,sum_count_y,time_y,sum_time
0,2022-01-03,712,3.560,97,1.2125,4.7725
1,2022-01-04,497,2.485,61,0.7625,3.2475
2,2022-01-05,669,3.345,31,0.3875,3.7325
3,2022-01-06,527,2.635,42,0.5250,3.1600
4,2022-01-07,585,2.925,44,0.5500,3.4750
...,...,...,...,...,...,...
362,2023-07-25,2479,12.395,125,1.5625,13.9575
363,2023-07-26,1601,8.005,49,0.6125,8.6175
364,2023-07-27,2650,13.250,82,1.0250,14.2750
365,2023-07-28,2462,12.310,71,0.8875,13.1975


In [180]:
combined_time = combined_time.drop(combined_time.columns[[1,2,3,4]], axis=1)
combined_time

,Order_date,sum_time
0,2022-01-03,4.7725
1,2022-01-04,3.2475
2,2022-01-05,3.7325
3,2022-01-06,3.1600
4,2022-01-07,3.4750
...,...,...
362,2023-07-25,13.9575
363,2023-07-26,8.6175
364,2023-07-27,14.2750
365,2023-07-28,13.1975


In [181]:
combined_time = pd.merge(combined_time, new_Not_OSR_time_single, on='Order_date')
combined_time

,Order_date,sum_time,sum_count,time
0,2022-01-03,4.7725,834,20.850
1,2022-01-04,3.2475,541,13.525
2,2022-01-05,3.7325,463,11.575
3,2022-01-06,3.1600,390,9.750
4,2022-01-07,3.4750,425,10.625
...,...,...,...,...
361,2023-07-25,13.9575,905,22.625
362,2023-07-26,8.6175,707,17.675
363,2023-07-27,14.2750,765,19.125
364,2023-07-28,13.1975,803,20.075


In [182]:
combined_time['Time'] = combined_time['sum_time'] + combined_time['time']
combined_time = combined_time.drop(combined_time.columns[[1,2,3]], axis=1)
combined_time


,Order_date,Time
0,2022-01-03,25.6225
1,2022-01-04,16.7725
2,2022-01-05,15.3075
3,2022-01-06,12.9100
4,2022-01-07,14.1000
...,...,...
361,2023-07-25,36.5825
362,2023-07-26,26.2925
363,2023-07-27,33.4000
364,2023-07-28,33.2725


In [ ]:
# Drop column 4,5,6,7,8,9,12
# Drop columns with Nb_of_colli & Nb_of_pallets == 0
# Share full case and single unit
# Count single unit according to their lines
# Count full case according to their lines
# Sum the counts by the unique date for full case and single unit
# calculate time for full case and single unit

## Outbound Data Repair

In [ ]:
# Convert the 'Date' column to a datetime object
merged_df['Order_date'] = pd.to_datetime(merged_df['Order_date']).dt.strftime('%d-%m-%Y')

In [ ]:
merged_df.head()

In [183]:
# Group date by Colli
new_colli = merged_df.groupby(['Order_number', 'Order_date', 'Nb_of_colli']).size().reset_index(name='count')

unique_order_number_colli = new_colli.drop_duplicates(subset=['Order_number'])
# new_colli.head()

# Group date by Palette
new_palette = merged_df.groupby(['Order_number', 'Order_date', 'Nb_of_pallets']).size().reset_index(name='count')

unique_order_number_pal = unique_order_numbers = new_palette.drop_duplicates(subset=['Order_number'])


In [192]:
# Check for duplicate order numbers in Palette and Colli
are_order_nums_unique = not unique_order_number_colli['Order_number'].duplicated().any()

if are_order_nums_unique:
    print("All order numbers are unique.")
else:
    print("There are duplicate order numbers.")

All order numbers are unique.


In [193]:
unique_order_number_pal

,Order_number,Order_date,Nb_of_pallets,count
0,00000001,2023-04-19,0.0,1
1,00082208,2022-08-11,1.0,11
2,0080000239,2022-05-04,32.0,6
3,0080000240,2022-05-04,20.0,4
4,0080000257,2022-06-15,11.0,2
...,...,...,...,...
273863,99000668,2023-07-12,0.0,1
273864,99000669,2023-07-14,0.0,1
273865,99000670,2023-07-31,0.0,1
273866,99000671,2023-07-25,0.0,1


In [194]:
new_drop = [0,3]

# unique_order_number_pal = unique_order_number_pal[unique_order_number_pal['Nb_of_pallets'] != 0].drop(unique_order_number_pal.columns[new_drop], axis=1)
unique_order_number_pal = unique_order_number_pal.drop(unique_order_number_pal.columns[new_drop], axis=1)

# unique_order_number_colli = unique_order_number_colli[unique_order_number_colli['Nb_of_colli'] != 0].drop(unique_order_number_colli.columns[new_drop], axis=1)
unique_order_number_colli = unique_order_number_colli.drop(unique_order_number_colli.columns[new_drop], axis=1)

In [ ]:
print(f'The dimension of the Colli dataset is {unique_order_number_colli.shape}')
print(f'The dimension of the Palette dataset is {unique_order_number_pal.shape}')

In [195]:
unique_order_number_pal = unique_order_number_pal.groupby('Order_date')['Nb_of_pallets'].sum().reset_index()

unique_order_number_colli = unique_order_number_colli.groupby('Order_date')['Nb_of_colli'].sum().reset_index()

In [197]:
unique_order_number_colli.sort_values(by='Order_date', ascending=True)

,Order_date,Nb_of_colli
0,2021-01-27,7.0
1,2021-03-11,15.0
2,2022-01-01,2.0
3,2022-01-03,1852.0
4,2022-01-04,1200.0
...,...,...
472,2023-07-26,1499.0
473,2023-07-27,3108.0
474,2023-07-28,1814.0
475,2023-07-29,14.0
